In [1]:
import pandas as pd
import os

In [2]:
COMBINED_DATA = os.path.join('..', 'data', 'combined-data.csv')

In [3]:
songs_df = pd.read_csv(COMBINED_DATA)
songs_df.head()

,Year,Artist,Title,URL,TrackId,Acousticness,Danceability,Duration (ms),Energy,Instrumentalness,...,Mode,Speechiness,Tempo,Time Signature,Valence,Album,Image,Explicit,Popularity,Winner
0,1958,Domenico Modugno,Nel Blu Dipinto Di Blu (Volare),https://open.spotify.com/track/006Ndmw2hHxvnLb...,006Ndmw2hHxvnLbJsBFnPx,0.987,0.518,216373.0,0.060,0.000008,...,1.0,0.0441,127.870,4.0,0.336,Tutto Modugno (Mister Volare),https://i.scdn.co/image/5e8c49f7a8d161c1d65109...,False,35.0,1
1,1958,Henry Mancini,The Music From Peter Gunn,https://open.spotify.com/track/3BdPP6Xce6FUcfa...,3BdPP6Xce6FUcfaCFsnZIg,0.893,0.550,177733.0,0.318,0.881000,...,1.0,0.0313,138.037,4.0,0.620,The music from Peter Gunn,https://i.scdn.co/image/1ad2e8ce1f988c27678298...,False,16.0,1
2,1958,"Domenico Modugno, songwriter (Domenico Modugno)",Nel Blu Dipinto Di Blu (Volare),https://open.spotify.com/track/006Ndmw2hHxvnLb...,006Ndmw2hHxvnLbJsBFnPx,0.987,0.518,216373.0,0.060,0.000008,...,1.0,0.0441,127.870,4.0,0.336,Tutto Modugno (Mister Volare),https://i.scdn.co/image/5e8c49f7a8d161c1d65109...,False,35.0,1
3,1958,Ella Fitzgerald,Ella Fitzgerald Sings The Irving Berlin Song Book,https://open.spotify.com/track/5FY0EikZVSBOwpj...,5FY0EikZVSBOwpjQa9S5Ii,0.675,0.615,138320.0,0.186,0.000000,...,0.0,0.0508,73.007,4.0,0.749,Ella Fitzgerald Sings The Irving Berlin Song Book,https://i.scdn.co/image/3350581fb4712a44a6f6b5...,False,34.0,1
4,1958,Perry Como,Catch A Falling Star,https://open.spotify.com/track/38YMdelhj62vJ6d...,38YMdelhj62vJ6d5a0wxMQ,0.864,0.628,148493.0,0.172,0.000000,...,1.0,0.0600,118.895,3.0,0.475,It's Impossible,https://i.scdn.co/image/0a33dd0dbd1b416c245c8c...,False,44.0,1


In [4]:
songs_df.columns

Index(['Year', 'Artist', 'Title', 'URL', 'TrackId', 'Acousticness',
       'Danceability', 'Duration (ms)', 'Energy', 'Instrumentalness', 'Key',
       'Liveness', 'Loudness', 'Mode', 'Speechiness', 'Tempo',
       'Time Signature', 'Valence', 'Album', 'Image', 'Explicit', 'Popularity',
       'Winner'],
      dtype='object')

In [5]:
X = songs_df.drop(labels=['Artist', 'Title', 'URL', 'TrackId', 'Album', 'Image', 'Winner'], axis=1)
X.head()

,Year,Acousticness,Danceability,Duration (ms),Energy,Instrumentalness,Key,Liveness,Loudness,Mode,Speechiness,Tempo,Time Signature,Valence,Explicit,Popularity
0,1958,0.987,0.518,216373.0,0.060,0.000008,10.0,0.1610,-14.887,1.0,0.0441,127.870,4.0,0.336,False,35.0
1,1958,0.893,0.550,177733.0,0.318,0.881000,0.0,0.0311,-14.516,1.0,0.0313,138.037,4.0,0.620,False,16.0
2,1958,0.987,0.518,216373.0,0.060,0.000008,10.0,0.1610,-14.887,1.0,0.0441,127.870,4.0,0.336,False,35.0
3,1958,0.675,0.615,138320.0,0.186,0.000000,0.0,0.1940,-12.382,0.0,0.0508,73.007,4.0,0.749,False,34.0
4,1958,0.864,0.628,148493.0,0.172,0.000000,11.0,0.1130,-17.816,1.0,0.0600,118.895,3.0,0.475,False,44.0


In [6]:
X.dtypes

Year                  int64
Acousticness        float64
Danceability        float64
Duration (ms)       float64
Energy              float64
Instrumentalness    float64
Key                 float64
Liveness            float64
Loudness            float64
Mode                float64
Speechiness         float64
Tempo               float64
Time Signature      float64
Valence             float64
Explicit               bool
Popularity          float64
dtype: object

In [7]:
X.Explicit = X.Explicit.astype(int)

In [8]:
X.Explicit.value_counts()

0    12983
1     1061
Name: Explicit, dtype: int64

In [9]:
#excluded_features = ['Popularity']
excluded_features = ['Key']
X.drop(labels=excluded_features, axis=1, inplace=True)

In [10]:
y = songs_df['Winner']
y.head()

0    1
1    1
2    1
3    1
4    1
Name: Winner, dtype: int64

In [11]:
y.value_counts()

0    10754
1     3290
Name: Winner, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, Normalizer, StandardScaler
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, shuffle=True, stratify=y)

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Convert labels to one-hot-encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

Using TensorFlow backend.
/Users/kishore/Tools/anaconda3/envs/PythonStuff/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/kishore/Tools/anaconda3/envs/PythonStuff/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/Users/kishore/Tools/anaconda3/envs/PythonStuff/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [13]:
X.shape

(14044, 15)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

dims = len(X.columns)
# Create model and add layers
def make_model(dense_layer_sizes, activation):
    model = Sequential()
    for dense_layer in dense_layer_sizes:
        model.add(Dense(units=dense_layer, activation=activation, input_dim=dims))
        
    model.add(Dense(units=2, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
dense_size_candidates = [[110, 110]]
classifier = KerasClassifier(make_model)
validator = GridSearchCV(classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     'epochs': [120],
                                     'activation': ['relu'],
                                    }, n_jobs=1)
validator.fit(X_train_scaled, y_train_categorical, validation_data=(X_test, y_test_categorical))



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/Users/kishore/Tools/anaconda3/envs/PythonStuff/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Train on 7022 samples, validate on 3511 samples
Epoch 1/120
7022/7022 [==============================] - 1s 92us/step - loss: 0.4673 - acc: 0.8033 - val_loss: 3.7736 - val_acc: 0.7659
Epoch 2/120
7022/7022 [==============================] - 0s 47us/step - loss: 0.4460 - acc: 0.8085 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 3/120
7022/7022 [==============================] - 0s 47us/step - loss: 0.4347 - acc: 0.8132 - val_loss: 3.7736 - val_acc: 0.7659
Epoch 4/120
7022/7022 [==============================] - 0s 49us/step - loss: 0.4267 - acc: 0.8157 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 5/120
7022/7022 [==============================] - 0s 46us/step - loss: 0.4204 - acc: 0.8173 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 6/120
7022/7022 [==============================] - 0s 46us/step - loss: 0.4170 - acc: 0.8204 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 7/120
7022/7022 [==============================] - 0s 48us/step - loss: 0.4132 - acc: 0.8250 - val_loss: 12.3399 - val_acc: 0.

7022/7022 [==============================] - 0s 40us/step - loss: 0.1571 - acc: 0.9433 - val_loss: 3.7736 - val_acc: 0.7659
Epoch 61/120
7022/7022 [==============================] - 0s 42us/step - loss: 0.1512 - acc: 0.9437 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 62/120
7022/7022 [==============================] - 0s 42us/step - loss: 0.1488 - acc: 0.9452 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 63/120
7022/7022 [==============================] - 0s 40us/step - loss: 0.1439 - acc: 0.9470 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 64/120
7022/7022 [==============================] - 0s 40us/step - loss: 0.1406 - acc: 0.9504 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 65/120
7022/7022 [==============================] - 0s 41us/step - loss: 0.1423 - acc: 0.9475 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 66/120
7022/7022 [==============================] - 0s 42us/step - loss: 0.1430 - acc: 0.9477 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 67/120
7022/7022 [========================

7022/7022 [==============================] - 0s 45us/step - loss: 0.0594 - acc: 0.9785 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 120/120
7022/7022 [==============================] - 0s 12us/step
Train on 7022 samples, validate on 3511 samples
Epoch 1/120
7022/7022 [==============================] - 1s 87us/step - loss: 0.4683 - acc: 0.8002 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 2/120
7022/7022 [==============================] - 0s 44us/step - loss: 0.4421 - acc: 0.8122 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 3/120
7022/7022 [==============================] - 0s 44us/step - loss: 0.4331 - acc: 0.8143 - val_loss: 3.7690 - val_acc: 0.7662
Epoch 4/120
7022/7022 [==============================] - 0s 44us/step - loss: 0.4253 - acc: 0.8190 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 5/120
7022/7022 [==============================] - 0s 44us/step - loss: 0.4195 - acc: 0.8191 - val_loss: 3.7736 - val_acc: 0.7659
Epoch 6/120
7022/7022 [==============================] - 0s 44us/step - 

...

7022/7022 [==============================] - 0s 48us/step - loss: 0.1653 - acc: 0.9416 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 55/120
7022/7022 [==============================] - 0s 46us/step - loss: 0.1593 - acc: 0.9420 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 56/120
7022/7022 [==============================] - 0s 46us/step - loss: 0.1512 - acc: 0.9436 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 57/120
7022/7022 [==============================] - 0s 47us/step - loss: 0.1489 - acc: 0.9459 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 58/120
7022/7022 [==============================] - 0s 46us/step - loss: 0.1536 - acc: 0.9437 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 59/120
7022/7022 [==============================] - 0s 49us/step - loss: 0.1441 - acc: 0.9487 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 60/120
7022/7022 [==============================] - 0s 48us/step - loss: 0.1398 - acc: 0.9514 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 61/120
7022/7022 [========================

7022/7022 [==============================] - 0s 48us/step - loss: 0.1055 - acc: 0.9644 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 114/120
7022/7022 [==============================] - 0s 45us/step - loss: 0.0748 - acc: 0.9756 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 115/120
7022/7022 [==============================] - 0s 47us/step - loss: 0.0411 - acc: 0.9903 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 116/120
7022/7022 [==============================] - 0s 46us/step - loss: 0.0346 - acc: 0.9929 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 117/120
7022/7022 [==============================] - 0s 47us/step - loss: 0.0276 - acc: 0.9953 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 118/120
7022/7022 [==============================] - 0s 46us/step - loss: 0.0305 - acc: 0.9936 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 119/120
7022/7022 [==============================] - 0s 48us/step - loss: 0.0297 - acc: 0.9943 - val_loss: 12.3399 - val_acc: 0.2344
Epoch 120/120
7022/7022 [================

Epoch 51/120
10533/10533 [==============================] - 1s 49us/step - loss: 0.2049 - acc: 0.9183 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 52/120
10533/10533 [==============================] - 1s 50us/step - loss: 0.2062 - acc: 0.9168 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 53/120
10533/10533 [==============================] - 1s 51us/step - loss: 0.2014 - acc: 0.9199 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 54/120
10533/10533 [==============================] - 0s 47us/step - loss: 0.1945 - acc: 0.9240 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 55/120
10533/10533 [==============================] - 0s 46us/step - loss: 0.1903 - acc: 0.9277 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 56/120
10533/10533 [==============================] - 0s 47us/step - loss: 0.1911 - acc: 0.9249 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 57/120
10533/10533 [==============================] - 0s 47us/step - loss: 0.1838 - acc: 0.9304 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 58/120
10533/10533 [=

Epoch 110/120
10533/10533 [==============================] - 0s 47us/step - loss: 0.0638 - acc: 0.9789 - val_loss: 3.7736 - val_acc: 0.7659
Epoch 111/120
10533/10533 [==============================] - 1s 48us/step - loss: 0.0640 - acc: 0.9803 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 112/120
10533/10533 [==============================] - 0s 45us/step - loss: 0.0725 - acc: 0.9747 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 113/120
10533/10533 [==============================] - 0s 46us/step - loss: 0.0666 - acc: 0.9781 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 114/120
10533/10533 [==============================] - 1s 56us/step - loss: 0.0557 - acc: 0.9823 - val_loss: 3.7736 - val_acc: 0.7659
Epoch 115/120
10533/10533 [==============================] - 1s 48us/step - loss: 0.0612 - acc: 0.9823 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 116/120
10533/10533 [==============================] - 0s 46us/step - loss: 0.0636 - acc: 0.9785 - val_loss: 3.7782 - val_acc: 0.7656
Epoch 117/120
10533/

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x1229417f0>,
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'dense_layer_sizes': [[110, 110]], 'epochs': [120], 'activation': ['relu']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
print('The parameters of the best model are: ')
print(validator.best_params_)
print(validator.best_score_)

The parameters of the best model are: 
{'activation': 'relu', 'dense_layer_sizes': [110, 110], 'epochs': 120}
0.7719548087191132


In [17]:
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(X_test_scaled, y_test_categorical)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

3511/3511 [==============================] - 0s 19us/step
loss :  1.2409757625951416
acc :  0.768157220317984


In [18]:
model_file = os.path.join('..', 'model', 'grammy_prediction_model.h5')
best_model.save(model_file)


In [19]:
X_test.columns


Index(['Year', 'Acousticness', 'Danceability', 'Duration (ms)', 'Energy',
       'Instrumentalness', 'Liveness', 'Loudness', 'Mode', 'Speechiness',
       'Tempo', 'Time Signature', 'Valence', 'Explicit', 'Popularity'],
      dtype='object')

In [20]:
from sklearn.externals import joblib
scaler_file = os.path.join('..', 'model', 'grammy_prediction_scaler.sav')
joblib.dump(X_scaler, scaler_file) 

['../model/grammy_prediction_scaler.sav']